*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
print(tf.__version__)
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding, LSTM, Dropout, Dense, Input

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
dataFrame_train = pd.read_csv(train_file_path, sep="\t", header=None, names=['label', 'text'])
dataFrame_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

In [ ]:
label_train = dataFrame_train.label.astype('category').cat.codes
label_test = dataFrame_test.label.astype('category').cat.codes

In [ ]:
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
stopwords_eng = set(stopwords.words('english'))
lematizador = WordNetLemmatizer()

In [ ]:
def cleanText(text):
    text = re.sub(r"([a-z0-9.\-+_]+@[a-z0-9.\-+_]+\.[a-z]+)|[^\w\s]", r"\1", text)
    text = " ".join([lematizador.lemmatize(word) for word in text.split()
                    if not word in stopwords_eng])
    text = text.lower()
    return text

In [ ]:
text_train = dataFrame_train['text'].apply(lambda x: cleanText(x))

In [ ]:
max_words = 1000
max_len = 500

In [ ]:
tokenWords = Tokenizer(num_words=max_words)
tokenWords.fit_on_texts(text_train)

In [ ]:
sequences = tokenWords.texts_to_sequences(text_train)
sequences[:5]

In [ ]:
sequences_matrix = pad_sequences(sequences, maxlen=max_len)
sequences_matrix[:5]

In [ ]:
model = Sequential()
model.add(Input(shape=[max_len]))
model.add(Embedding(max_words, 50, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy']
)
model.summary()

In [ ]:
r = model.fit(sequences_matrix, label_train, batch_size=128, epochs=5, validation_split=0.2)

In [ ]:
def preprocessing(X):
  x = X.apply(lambda x: cleanText(x))
  x = tokenWords.texts_to_sequences(x)
  return pad_sequences(x, maxlen=max_len)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  modelPrediction = model.predict(preprocessing(pd.Series([pred_text])))[0]
  if modelPrediction.all() < 0.51:
    prediction = "ham"
  else:
    prediction = "spam"
  return (modelPrediction[0], prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
